In [95]:
import requests as rq
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep

In [96]:
def click_show_more(driver):
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="c8hZ6"]/div/div'))
        )

        # Scroll to the "Show More" button to make it clickable
        ActionChains(driver).move_to_element(show_more_button).perform()
        time.sleep(1)  # Add a small delay to ensure the button is clickable

        # Click the "Show More" button
        show_more_button.click()

        # You may need to wait for the new content to load before proceeding
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.XPATH, 'loading_indicator_xpath')))
        
    except Exception as e:
        print(f"An error occurred while clicking 'Show More': {e}")


In [97]:
sources=['CMN','CMN','CMN']
destinations=['PAR','ROM','IST']
classes=['economy','business']

# Features

In [98]:
def get_airline(soup):
    #get airlines
    airlines=[]
    items=soup.find_all('div',class_='J0g6-operator-text') 
    for item in items :
        airl=item.text
        airl = airl.split(',')[0].strip()
        airlines.append(airl)
    
    return airlines

In [99]:
def get_duration(soup):
    #getduration
    durations=[]
    duration_div=soup.find_all('div',class_='xdW8 xdW8-mod-full-airport')
    for d in duration_div :
        duration=d.find('div',class_='vmXl vmXl-mod-variant-default').text
        durations.append(duration)
    return durations

In [100]:
def get_stops(soup):
    stops=[]
    stop_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-default')
    for s in stop_div :
        selected_element = s.select_one('.JWEO-stops-text, .JWEO-mod-warning')
        if selected_element:
          stops.append(selected_element.text)

    return stops

In [101]:
def get_price(soup):   
    #getprice
    prices=[]
    price_div=soup.find_all('div',class_='f8F1-price-text-container')

    for p in price_div :
        price=p.find('div',class_='f8F1-price-text').text
        price= price.replace('\xa0', ' ')
        prices.append(price)
    return prices

In [102]:
def get_class(soup):
    #getclass
    classes=[]
    class_div=soup.find_all('div',class_='aC3z-option')
    for c in class_div :
        classe=c.find('div',class_='aC3z-name').text
        classes.append(classe)
    return classes

In [103]:
#get_depature_time
def get_dep_time(soup):
    deps=[]
    dep_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for d in dep_div:
        depature=d.find_all('span')
        deps.append(depature[0].text)
    return deps
#get_arrival_time
def get_arr_time(soup):
    arrs=[]
    arr_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for a in arr_div:
        arrival=a.find_all('span')
        arrs.append(arrival[2].text)
    return arrs


# Scraping economy class

In [105]:
df=pd.DataFrame(columns=["Airline", "Source", "Destination","Duration","stops","class","depature time","arrival time", "Price","Date"])

for j in range(len(classes)):    
    for i in range(len(sources)): 
        driver = webdriver.Edge()
        date='2023-12-04'
        kayak=f"https://www.kayak.ae/flights/{sources[i]}-{destinations[i]}/{date}/{classes[j]}?sort=bestflight_a"
        print(sources[i])
        print(destinations[i])
        driver.get(kayak)
        # Click "Show More" button
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ULvh-button.show-more-button')))
        count=5
        while count>1:
            try:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ULvh-button.show-more-button'))).click()
                print("MORE button clicked")
            except TimeoutException:
                break
            count-=1
            sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airline(soup)
        print(len(airlines))
        stops = get_stops(soup)
        print(len(stops))
        dep_time=get_dep_time(soup)
        print(len(dep_time))
        arr_time =get_arr_time(soup)
        print(len(arr_time))
        prices = get_price(soup)
        print(len(prices))
        duration = get_duration(soup)
        print(len(duration))
        df = pd.concat([df,pd.DataFrame({
                        'Airline': airlines,
                        'Duration': duration,
                        'stops' : stops,
                        'class' : classes[j],
                        'Source':sources[i],
                        'Destination': destinations[i],
                        'depature time' : dep_time ,
                        'arrival time' : arr_time,
                        'Price' : prices,
                        'Date' : date})])

        
        df = df.replace('\n','', regex=True)
        #df = df.reset_index(drop = True)
        # save data as csv file for each route
    
        print(f"Succesfully saved {sources[i]} => {destinations[i]} route  ")
        driver.quit()
df.to_csv('airlines_dataset.csv',index=False)
driver.quit()

CMN
PAR
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
75
75
75
75
75
75
Succesfully saved CMN => PAR route  
CMN
ROM
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
76
76
76
76
76
76
Succesfully saved CMN => ROM route  
CMN
IST
MORE button clicked
MORE button clicked
45
45
45
45
45
45
Succesfully saved CMN => IST route  
CMN
PAR
MORE button clicked
MORE button clicked
MORE button clicked
52
52
52
52
52
52
Succesfully saved CMN => PAR route  
CMN
ROM
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
72
72
72
72
72
72
Succesfully saved CMN => ROM route  
CMN
IST
MORE button clicked
MORE button clicked
41
41
41
41
41
41
Succesfully saved CMN => IST route  
